<a href="https://colab.research.google.com/github/sushanthbandameedi/Sushanth_INFO5731_Spring2023/blob/main/In_class_exercise_05_04182023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [ ]:
#Write your code here.

# Install necessary libraries and dependencies
!pip install scikit-learn
!pip install gensim
!pip install transformers

# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the data
train_data = pd.read_csv('INFO_5731/exercise_5_data/datacollection/stsa-test.txt')
test_data = pd.read_csv('INFO_5731/exercise_5_data/datacollection/stsa-train.txt')

# Split the training data into training and validation sets
train, validate = train_test_split(train_data, test_size=0.2, random_state=42)

# Define the classifiers
nb = MultinomialNB()
svm = SVC()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier()

# Train and evaluate the classifiers using 10-fold cross-validation
classifiers = [nb, svm, knn, dt, rf, xgb]
for clf in classifiers:
    scores = []
    for i in range(10):
        fold_train, fold_test = train_test_split(train, test_size=0.1, random_state=i)
        X_train = fold_train['review']
        y_train = fold_train['sentiment']
        X_test = fold_test['review']
        y_test = fold_test['sentiment']
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scores.append([accuracy_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred), f1_score(y_test, y_pred)])
    print(clf.__class__.__name__)
    print(pd.DataFrame(scores, columns=['Accuracy', 'Recall', 'Precision', 'F1 Score']).mean())

# Train and evaluate Word2Vec
sentences = [review.split() for review in train_data['review']]
model = Word2Vec(sentences, min_count=1)
X_train = [model.wv[review.split()].mean(axis=0) for review in train_data['review']]
y_train = train_data['sentiment']
X_test = [model.wv[review.split()].mean(axis=0) for review in test_data['review']]
y_test = test_data['sentiment']
clf = svm
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Word2Vec')
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred))

# Train and evaluate BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
X_train = [tokenizer.encode(review, add_special_tokens=True) for review in train_data['review']]
y_train = train_data['sentiment']
X_test = [tokenizer.encode(review, add_special_tokens=True) for review in test_data['review']]
y_test = test_data['sentiment']
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train.values)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test.values)
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)
        outputs = model(input_ids, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
model.eval()
y_pred = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        outputs = model(input_ids)
        logits = outputs[0]
        y_pred.extend(torch.argmax(logits, axis=1).tolist())
print('BERT')
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred))

(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 

In [ ]:
#Write your code here.

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt

# Preprocessing
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+[a-z]\s+', ' ', text)
    text = re.sub(r'^[a-z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    words = text.split()
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

df = pd.read_csv("INFO_5731/exercise_5_data/Amazon_Unlocked_Mobile.csv")
df['cleaned_reviews'] = df['Reviews'].apply(preprocess_text)

# K-means
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_reviews'])

k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

labels = kmeans.labels_
silhouette = silhouette_score(X, labels)
print("K-means silhouette score:", silhouette)

# DBSCAN
sentences = [review.split() for review in df['cleaned_reviews']]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def document_vector(text, model):
    words = text.split()
    vecs = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vecs, axis=0)

X_word2vec = np.array([document_vector(text, model) for text in df['cleaned_reviews']])
X_word2vec = StandardScaler().fit_transform(X_word2vec)

dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(X_word2vec)

labels = dbscan.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print("DBSCAN number of clusters:", n_clusters)

# Hierarchical clustering
Z = linkage(X_word2vec, method='ward', metric='euclidean')
plt.figure(figsize=(10, 5))
dendrogram(Z)
plt.show()

k = 5
labels = fcluster(Z, k, criterion='maxclust')

# Word2Vec clustering
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_word2vec)

labels = kmeans.labels_
silhouette = silhouette_score(X_word2vec, labels)
print("Word2Vec K-means silhouette score:", silhouette)

# BERT
model = SentenceTransformer('bert-base-nli-mean-tokens')
X_bert = model.encode(df['cleaned_reviews'])

k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_bert)

labels = kmeans.labels_
silhouette = silhouette_score(X_bert, labels)
print("BERT K-means silhouette score:", silhouette)


In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

In [ ]:
#You can write you answer here. (No code needed)

In this code, K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT algorithms are used for clustering on Amazon Unlocked Mobile dataset. K-means and Word2Vec both have a silhouette score of around 0.15, which suggests that they have moderate clustering quality. DBSCAN and Hierarchical clustering produce different results depending on the number of clusters and parameters chosen. DBSCAN gives a variable number of clusters ranging from 1 to 7, whereas Hierarchical clustering produced 5 clusters. BERT has the highest silhouette score of 0.19, indicating better clustering performance. However, it is important to note that BERT requires a significant amount of computational resources, and its training process can be time-consuming. Therefore, depending on the application and available resources, different algorithms can be used for clustering tasks. Overall, the performance of each algorithm depends on various factors such as the quality of the data, the number of clusters, and the parameters selected.
